In [24]:
import pandas as pd

# Target columns for consistency
target_columns = [ 
    "Restaurant Name", "Location", "Cuisine", "Price", 
    "Ratings(Dining, Delivery)", "Operating Hours", 
    "Contact", "Menu", "More Info"
]

# Load CSVs with pandas (since CSVLoader doesn’t handle merging well)
df1 = pd.read_csv("restaurants.csv")
df2 = pd.read_csv("restaurants_swiggy.csv")

# Merge the two datasets on common columns (Restaurant Name + Location is a good key)
merged_df = pd.merge(df1, df2, on=["Restaurant Name", "Location"], how="outer", suffixes=('_1', '_2'))

# Resolve column conflicts (fill missing values where appropriate)
for col in ["Cuisine", "Price", "Ratings(Dining, Delivery)", "Operating Hours", "More Info"]:
    merged_df[col] = merged_df[f"{col}_1"].combine_first(merged_df[f"{col}_2"])
    merged_df.drop([f"{col}_1", f"{col}_2"], axis=1, inplace=True)

merged_df

,Restaurant Name,Location,Contact,Menu,Cuisine,Price,"Ratings(Dining, Delivery)",Operating Hours,More Info
0,34 Chowringhee Lane,"Shop No. 4, 7/120A, Swaroop Nagar, Kanpur, Utt...",NaN,https://dineout-media-assets.swiggy.com/swiggy...,"Teekha Hitchki Chicken Roll, Karara Aloo Roll,...",₹500 for two,5 • 2 in Google •,"CLOSED, OPENS AT 12PM","Parking available, SwiggyPay accepted"
1,360* By Lush,"Opp. Raj Ratan, Moti Jheel, Harsh Nagar Near M...",NaN,https://dineout-media-assets.swiggy.com/swiggy...,"North Indian, South Indian",₹500 for two,4.3 • 257 in Google •,"CLOSED, OPENS AT 11AM","Reservation available, Parking available, Smok..."
2,7th Heaven Cake And Cafe,"Plot No.120/70/A, Ground Floor, First Street, ...",NaN,https://dineout-media-assets.swiggy.com/swiggy...,Italian,₹1000 for two,4.1 • 52 in Google •,"CLOSED, OPENS AT 11:55AM","Reservation available, Parking available, Free..."
3,AD's Kitchen Bar,112/311 A Swaroop Nagar Kanpur Near Bachaulal ...,NaN,https://dineout-media-assets.swiggy.com/swiggy...,"Chinese, North Indian",₹1000 for two,4.3 • 29 in Google •,"CLOSED, OPENS AT 2PM","Reservation available, Parking available, Free..."
4,Almost Cafe,"Plot No.120/611A, Lower Ground Lajpat Nagar, A...",NaN,https://dineout-media-assets.swiggy.com/swiggy...,Continental,₹800 for two,4.3 • 528 in Google •,"CLOSED, OPENS AT 11AM","Reservation available, Free wifi, SwiggyPay ac..."
...,...,...,...,...,...,...,...,...,...
93,Velocity Restaurant & Bar,3/Pr-1 Ambedkarpuram Awas Vikas No-3 Kalyanpur...,NaN,https://dineout-media-assets.swiggy.com/swiggy...,Chinese,₹700 for two,4.2 • 297 in Google •,"CLOSED, OPENS AT 12PM","Alcohol served, Parking available, Smoking are..."
94,WOW! Momo,"Shop 1/13/301, Govind Nagar, Nandlal Chawraha,...",9.191984e+13,NaN,"Momos, Chinese, Fast Food, Asian, Beverages, F...",₹400 for two,"4.1, 4.1",9am – 5am (Today),"Home Delivery, Takeaway Available, Indoor Seating"
95,Waterview,"Moti Jheel, Harsh Nagar Near Musical Fountain ...",NaN,https://dineout-media-assets.swiggy.com/swiggy...,"Chinese, North Indian",₹1500 for two,-- • too few ratings •,"CLOSED, OPENS AT 12PM","Reservation available, Parking available, Vale..."
96,Yappy Cafe,"Moti Jheel, Opposite Passport Seva Kendra, Nav...",NaN,https://dineout-media-assets.swiggy.com/swiggy...,North Indian,₹700 for two,4.2 • 112 in Google •,"CLOSED, OPENS AT 12PM","Reservation available, Parking available, Pet ..."


In [25]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


In [31]:
import re
import requests
from PIL import Image
from io import BytesIO


def extract_text_from_image_urls(text):
    if pd.isna(text):
        return ""

    # Match full URLs ending in .jpg, .jpeg, or .png
    print(f"Extracting URLs from text: {text}")
    url_pattern = r'https?://[^\s]+?\.(?:jpg|jpeg|png)'
    urls = re.findall(url_pattern, text)
    extracted_texts = []

    for url in urls:
        try:
            response = requests.get(url, timeout=10)
            img = Image.open(BytesIO(response.content)).convert("RGB")
            text = pytesseract.image_to_string(img, lang="eng")
            extracted_texts.append(text.strip())
        except Exception as e:
            print(f"Error processing image URL {url}: {e}")
    raw_text = str(extracted_texts)
    # Normalize and clean symbols first
    text = raw_text.replace("\\n", "\n")  # in case it's stringified list
    text = re.sub(r'\[.*?\]', '', text)   # Remove things like [=], [4], etc.
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Strip non-ASCII
    text = re.sub(r'\s+', ' ', text).strip()

    # Replace common OCR misreads for ₹
    text = re.sub(r'[\~=fFtT@Z]', '₹', text)
    text = text.replace('a9', '₹89')  # Special case handling
    text = re.sub(r'₹{2,}', '₹', text)  # Collapse repeated ₹

    # Use regex to extract (item, price) patterns
    menu_items = []

    pattern = r'([A-Za-z0-9&\-\!\(\)\'\. ]{3,})\s+(?:₹)?(\d{2,4})'
    matches = re.findall(pattern, text)

    for name, price in matches:
        name = name.strip(" :;.,~₹").title()
        price = "₹" + price.strip()
        menu_items.append({"item": name, "price": price})

    return menu_items



# Apply the function to the "Menu" column only first row
merged_df["Menu Text"] = merged_df["Menu"].apply(extract_text_from_image_urls)
# Replace "Menu" column with extracted text for use in documents
merged_df["Menu"] = merged_df["Menu Text"]
merged_df.drop(columns=["Menu Text"], inplace=True)


merged_df.to_csv("merged_data.csv", index=False)

Extracting URLs from text: https://dineout-media-assets.swiggy.com/swiggy/image/upload/fl_lossy,f_auto,q_auto/v1676354419/020e0035e620a01dcf70eb113635d56a.jpg, https://dineout-media-assets.swiggy.com/swiggy/image/upload/fl_lossy,f_auto,q_auto/v1676354422/12795cb7ccc0c4a2214c32abed536360.jpg, https://dineout-media-assets.swiggy.com/swiggy/image/upload/fl_lossy,f_auto,q_auto/v1676354425/0075e85629f8edcc183b2a531eac478a.jpg
Extracting URLs from text: https://dineout-media-assets.swiggy.com/swiggy/image/upload/fl_lossy,f_auto,q_auto/DINEOUT_ALL_RESTAURANTS/IMAGES/RESTAURANT_IMAGE_SERVICE/2024/5/13/f82c841b-c308-4a25-bb92-15b362a0a4a8_20240513T113021580.jpg, https://dineout-media-assets.swiggy.com/swiggy/image/upload/fl_lossy,f_auto,q_auto/DINEOUT_ALL_RESTAURANTS/IMAGES/RESTAURANT_IMAGE_SERVICE/2024/5/13/8d445174-f270-41a5-9083-3e498b4ff52b_20240513T113021937.jpg, https://dineout-media-assets.swiggy.com/swiggy/image/upload/fl_lossy,f_auto,q_auto/DINEOUT_ALL_RESTAURANTS/IMAGES/RESTAURANT_IMA

In [ ]:
# from langchain.document_loaders.csv_loader import CSVLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# # Load documents from CSV
# loader1 = CSVLoader(file_path="restaurants.csv")
# loader2 = CSVLoader(file_path="restaurants_swiggy.csv")
# documents = loader1.load()+loader2.load()

# # Split documents into chunks for better retrieval
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200
# )
# chunks = text_splitter.split_documents(documents)

In [14]:
import ast
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings  # or HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import pickle
import pandas as pd

# ------------------ Step A: CSV Load ------------------
csv_path = "merged_data.csv"
df = pd.read_csv(csv_path)

# ------------------ Step B: Parse & Clean Menu JSON Strings ------------------
def parse_menu(menu_str):
    try:
        return ast.literal_eval(menu_str) if pd.notnull(menu_str) else []
    except Exception as e:
        print(f"Error parsing menu: {menu_str} \n{e}")
        return []

df['Parsed_Menu'] = df['Menu'].apply(parse_menu)

# ------------------ Step C: Rich Text Construction with Menu Items ------------------
def build_rich_text(row):
    name = row.get('Restaurant Name', 'Unknown')
    location = row.get('Location', '')
    contact = row.get('Contact', '')
    cuisine = row.get('Cuisine', '')
    price = row.get('Price', '')
    ratings = row.get('Ratings(Dining, Delivery)', '')
    hours = row.get('Operating Hours', '')
    info = row.get('More Info', '')

    # Format ratings
    rating_dining, rating_delivery = "N/A", "N/A"
    try:
        ratings_list = ast.literal_eval(ratings)
        if isinstance(ratings_list, list) and len(ratings_list) == 2:
            rating_dining, rating_delivery = ratings_list
    except:
        pass

    menu_text = ". ".join([f"{item['item']} for {item['price']}" for item in row['Parsed_Menu']])

    full_text = (
        f"{name} is located at {location}. "
        f"Cuisine: {cuisine}. Price range: {price}. "
        f"Ratings - Dining: {rating_dining}, Delivery: {rating_delivery}. "
        f"Contact: {contact}. Operating Hours: {hours}. "
        f"Additional Info: {info}. "
        f"Menu includes: {menu_text}."
    )
    return full_text

df['rich_text'] = df.apply(build_rich_text, axis=1)

# ------------------ Step D: LangChain Document Creation ------------------
documents = [
    Document(
        page_content=row['rich_text'],
        metadata={
            "restaurant_name": row['Restaurant Name'],
            "location": row['Location']
        }
    )
    for _, row in df.iterrows()
]

In [15]:
documents

[Document(metadata={'restaurant_name': '34 Chowringhee Lane', 'location': 'Shop No. 4, 7/120A, Swaroop Nagar, Kanpur, Uttar Pradesh-208002, Nagar Nigam Food Safety Zone-16, Kanpur Nagar, , Uttar Pradesh, 208002'}, page_content='34 Chowringhee Lane is located at Shop No. 4, 7/120A, Swaroop Nagar, Kanpur, Uttar Pradesh-208002, Nagar Nigam Food Safety Zone-16, Kanpur Nagar, , Uttar Pradesh, 208002. Cuisine: Teekha Hitchki Chicken Roll, Karara Aloo Roll, Chicken Keema Samosa Dhamaka Roll, Veg Jalfrezi Roll, Chicken Gravy Momos , Paneer Gravy Momos. Price range: ₹500 for two. Ratings - Dining: N/A, Delivery: N/A. Contact: nan. Operating Hours: CLOSED, OPENS AT 12PM. Additional Info: Parking available, SwiggyPay accepted. Menu includes: 9 X89 Lambu 109 for ₹119. 169 for ₹159. 169 169 2129 for ₹1692. 149 for ₹7169. Ip 2179 for ₹1469. U Lambu Egg Roll for ₹89. Double Egg Roll for ₹99. Classic Chicken Roll for ₹169. Chicken Seekh Roll for ₹7169. Garlic Chicken Roll for ₹179. Chill! Chicken Roll

In [16]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# split_docs = text_splitter.split_documents(documents)
# split_docs

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Initialize embeddings - using a strong open-source model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'}
)

# Create vector store
vector_store = FAISS.from_documents(documents, embeddings)

In [18]:
vector_store.similarity_search("which rasturants has south indian available like dosa", k=5)

[Document(id='39c9718e-28b8-4cb6-8585-89492d8a0fa6', metadata={'restaurant_name': 'Shor', 'location': '113/168, Kanpur Vidya Mandir, Khalasi Line, Swaroop Nagar Near Bagicha Blorist North Kanpur 208002'}, page_content="Shor is located at 113/168, Kanpur Vidya Mandir, Khalasi Line, Swaroop Nagar Near Bagicha Blorist North Kanpur 208002. Cuisine: Chinese, North Indian. Price range: ₹1500 for two. Ratings - Dining: N/A, Delivery: N/A. Contact: nan. Operating Hours: CLOSED, OPENS AT 2PM. Additional Info: Alcohol served, Reservation available, Parking available, SwiggyPay accepted. Menu includes: E Dining Lounge (Pdl) Minimum Billing - 15K Maximum Person - for ₹12. Pdl for ₹10. 15K & Above for ₹20. -S for ₹60. E Rum 325 Bacardi Black Rum 3258 Old Monk Dark Rum 325B We Sy for ₹60. Camino for ₹5258. All Sauza Silver 5508 Bloody Mary for ₹650. Y Do for ₹650. Drink for ₹120. H Crushed Ice Juice for ₹150. Caprioska for ₹650. Red Bull for ₹250. Coke 120 Moscow Mule for ₹650. Resh Lime for ₹175. L

In [19]:
import dotenv
import os
dotenv.load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
# from huggingface_hub import login
# HUGGINGFACEHUB_API_TOKEN= os.getenv("HUGGINGFACEHUB_API_TOKEN")
# login(HUGGINGFACEHUB_API_TOKEN)  # Call this before model loading


In [20]:
from langchain_groq.chat_models import ChatGroq

# Initialize the Groq LLM
llm = ChatGroq(
    model_name="llama3-8b-8192",  # You can also use "mixtral-8x7b-32768" or other models
    temperature=0.7,
    max_tokens=1024
)

In [21]:

# Basic retriever
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)


In [22]:
retriever.get_relevant_documents(" best rasturants?")

C:\Users\agarw\AppData\Local\Temp\ipykernel_30376\3652090905.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(" best rasturants?")


[Document(id='3d7e0a58-db28-41bb-b474-a7c2d0ab9df6', metadata={'restaurant_name': 'The Red Olive', 'location': 'M/S The Crystal Hotel - R.M. Heritage & Hotels(P) Ltd, 111/7A, Harsh Nagar, Benajhabar, Kanpur- 208012'}, page_content='The Red Olive is located at M/S The Crystal Hotel - R.M. Heritage & Hotels(P) Ltd, 111/7A, Harsh Nagar, Benajhabar, Kanpur- 208012. Cuisine: Asian, North Indian. Price range: ₹1200 for two. Ratings - Dining: N/A, Delivery: N/A. Contact: nan. Operating Hours: CLOSED, OPENS AT 11AM. Additional Info: Reservation available, Parking available, SwiggyPay accepted. Menu includes: S 350 350 350 250 250 250 250 250 for ₹250. Icuisine Salads Caesar Salad for ₹300. Greek Salad for ₹300. Salad for ₹300. Herbs And Spices. Kachumber Salad for ₹350. Imes Chili Peppers. Green Salad for ₹300. Ried Papad for ₹120. And Sour Soup (& for ₹250. Able Or Chicken. Manchow Soup ( for ₹250. Able Or Chicken. Lemon Coriander Soup for ₹250. Corn Soup for ₹250. Able Or Chicken) for ₹250. 

In [36]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
# Initialize memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"  
)

restaurant_qa_template = """
You are a knowledgeable restaurant recommendation assistant for Kanpur, India.
Answer the question briefly and helpfully based on the information below. Avoid repeating irrelevant context or chat history.

For restaurant recommendations, include if available:
- Name and location
- Price range
- Ratings
- Key features (e.g. outdoor seating, live music, romantic)
- Opening hours
- Contact info

Keep responses concise and directly relevant to the question. Do not mention the source or say things like "based on the data."

Context:
{context}

Chat History:
{chat_history}

Question: {question}
Answer:
"""

QA_PROMPT = PromptTemplate(
    template=restaurant_qa_template, 
    input_variables=["context", "question", "chat_history"]
)

# Update your chain with this prompt
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    verbose=True,
    combine_docs_chain_kwargs={"prompt": QA_PROMPT}
)


In [31]:
# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate

# # Prompt for transforming vague or follow-up queries using chat history
# query_transform_prompt = PromptTemplate(
#     input_variables=["question", "history_context"],
#     template="""Given the following user question about restaurants, reformulate it to be 
# more effective for retrieving relevant information from a database of restaurant details, 
# menus, and features. Just provide the reformulated query without any additional text.

# The goal is to make the query more specific and relevant to the context of restaurant recommendations.
# **Please take into account the previous conversation to rewrite vague follow-up queries.**
# Avoid adding assumptions.

# Conversation history:
# {history_context}

# Current user query: {question}
# Improved search query:"""
# )

# query_transform_chain = LLMChain(
#     llm=llm,
#     prompt=query_transform_prompt
# )

# # Function that reformulates user queries with context
# def transform_query(question, chat_turns):
#     # Extract last 2 exchanges from chat_turns
#     history_context = "\n".join(
#         [f"User: {u}\nBot: {b}" for u, b in chat_turns[-2:]]
#     )

#     response = query_transform_chain.run({
#         "question": question,
#         "history_context": history_context
#     })

#     # Safely extract if needed
#     return response


In [32]:
retriever.get_relevant_documents("Top South Indian restaurants in Kanpur, serving vegetarian options and with a rating of 4.5 or higher")

[Document(id='dd15cc02-f665-42f0-be92-087882d273bd', metadata={'restaurant_name': 'The Central Courtyard', 'location': '112/350, Block C, Plot 400, Gutaiya, Nagar Nigam Food Safety Zone 16, Swaroop Nagar Indian Overseas Bank-Swaroop Nagar Branch North Kanpur 208002'}, page_content="The Central Courtyard is located at 112/350, Block C, Plot 400, Gutaiya, Nagar Nigam Food Safety Zone 16, Swaroop Nagar Indian Overseas Bank-Swaroop Nagar Branch North Kanpur 208002. Cuisine: Continental, North Indian. Price range: ₹1000 for two. Ratings - Dining: N/A, Delivery: N/A. Contact: nan. Operating Hours: CLOSED, OPENS AT 12:30PM. Additional Info: Reservation available, Parking available, Free wifi, SwiggyPay accepted. Menu includes: Ull for ₹249. - Add Chicken for ₹49. 'Dou Naan Bombs for ₹249. Rani Soya Chaap for ₹269. Ikka Pyramids for ₹299. 7- Malai Soya Chaap for ₹299. Ikka for ₹349. Ikka for ₹329. Ikka for ₹349. Ikka for ₹369. Angri for ₹339. Angri for ₹359. Andoori for ₹399. ) Sikandari Shahn

In [ ]:
from langchain.schema.messages import BaseMessage

def get_recent_chat_history(n=3):
    # Get all messages
    history = memory.chat_memory.messages
    # Only keep the last n pairs (each turn is user + assistant = 2 messages)
    last_n = history[-(n*2):] if len(history) >= n*2 else history
    return "\n".join([f"{m.type.capitalize()}: {m.content}" for m in last_n if isinstance(m, BaseMessage)])

from langchain.schema.runnable import RunnableMap, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain

# --------------------------------------
# 🧠 Intent Classifier
# --------------------------------------
intent_prompt = PromptTemplate.from_template("""
You are an intent classifier for a restaurant chatbot. Given a user query, classify it into one of the following intents:
- "comparison": Only if the user wants to compare two or more restaurants
- "diet": if the user is asking about dietary needs, pricing, veg/non-veg, or food restrictions
- "qa": if the user is asking about restaurant features, menus, ratings, or contact info
- "fallback": if the question is unrelated or unclear

Respond with only one word: comparison, diet, qa, or fallback.Before classifying t fallback, check the chat history for context.    
                                             
Chat History:
{chat_history}

User Query: "{question}"
Intent:
""")
intent_chain = LLMChain(llm=llm, prompt=intent_prompt)
def classify_intent(question: str) -> str:
    chat_his=get_recent_chat_history(3)
    intent = intent_chain.run({"question": question, "chat_history": chat_his}).strip().lower()
    valid_intents = {"comparison", "diet", "qa", "fallback"}
    return intent if intent in valid_intents else "fallback"





# --------------------------------------
# 📊 Comparison Tool (Structured Prompt)
# --------------------------------------
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

compare_prompt = PromptTemplate.from_template("""
You are a restaurant analyst comparing menus and features. Answer based on:
- Food options (e.g., veg, spice, gluten-free)
- Menu variety
- Pricing differences
- Any contrast in services (e.g., ambience, hours)

Chat History:
{chat_history}

                                              
Use this data:
{context}

Question: {question}

Comparison:
""")

comparison_chain = LLMChain(llm=llm, prompt=compare_prompt)

def compare_agent(question):
    docs = retriever.get_relevant_documents(question)
    if not docs:
        return fallback_agent(question)
    ctx = "\n".join([doc.page_content for doc in docs])
    chat_his=get_recent_chat_history(3)
    return comparison_chain.run({"context": ctx, "question": question, "chat_history": chat_his})


# --------------------------------------
# 🥗 Diet/Price Filter
# --------------------------------------
diet_prompt = PromptTemplate.from_template("""
You are helping users find restaurants or items based on dietary needs or price.

Chat History:
{chat_history}


Context:
{context}
Question: {question}

Answer:
""")

diet_chain = LLMChain(llm=llm, prompt=diet_prompt)

def diet_agent(question):
    docs = retriever.get_relevant_documents(question)
    if not docs:
        return fallback_agent(question)
    ctx = "\n".join([doc.page_content for doc in docs])
    chat_his=get_recent_chat_history(3)
    return diet_chain.run({"context": ctx, "question": question, "chat_history": chat_his})

# --------------------------------------
#  FallBack Agent (General QA)
# --------------------------------------
def fallback_agent():
    return (
        "🤖 I'm here to help you with information about restaurants in Kanpur! "
        "If you have questions about menu items, prices, comparisons, or dietary options, "
        "feel free to ask. 😊"
    )



# --------------------------------------
# 🎛️ Dispatcher (Intent Router)
# --------------------------------------
def route_agent(question):
    print(f"User: {question}")
    intent = classify_intent(question)
    print(f"Intent: {intent}")

    if intent == "qa":
        result = qa_chain({"question": question, "chat_history": get_recent_chat_history})
        answer = result["answer"]
    elif intent == "comparison":
        answer = compare_agent(question)
    elif intent == "diet":
        answer = diet_agent(question)
    else:
        answer = fallback_agent()

    memory.save_context({"question": question}, {"answer": answer})

    return answer





# --------------------------------------
# 💬 Chat Loop (Test It!)
# --------------------------------------
def chat_loop():
    print("🤖 Ask anything about Kanpur restaurants!")
    while True:
        user_input = input("🧠 You: ")
        if user_input.lower() in ["exit", "quit"]:
            break
        try:
            response = route_agent(user_input)
            print(f"🤖 Bot: {response}\n")
        except Exception as e:
            print(f"⚠️ Error: {e}")

if __name__ == "__main__":
    chat_loop()

🤖 Ask anything about Kanpur restaurants!
User: What are some good open-air bars in Kanpur with a chill vibe?
Intent: qa


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a knowledgeable restaurant recommendation assistant for Kanpur, India.
Answer the question briefly and helpfully based on the information below. Avoid repeating irrelevant context or chat history.

For restaurant recommendations, include if available:
- Name and location
- Price range
- Ratings
- Key features (e.g. outdoor seating, live music, romantic)
- Opening hours
- Contact info

Keep responses concise and directly relevant to the question. Do not mention the source or say things like "based on the data."

Context:
Offline Cafe is located at 10/503, Khalasi Line, Tilak Nagar, Kanpur, Khalasi Line Nu York Home Decor North Kanpur 208002. Cuisine: Continental, Italian. Price range: ₹1000 for two. Ratings - Dining: N/A, Delivery: N/A. Contact: nan. Opera